# GWAS Tutorial for Hail-jp hands on seminar

このノートブックは、HailのドキュメントにあるGenome-Wide Association Study(GWAS) Tutorial(https://hail.is/docs/0.2/tutorials-landing.html) をベースに、Hail-jpのハンズオンセミナーのために加筆したものです。

遺伝子データセットを操作および探索する機能に重点を置いて、Hailの機能の概要を習得できるように設計されています。 
また、GWAS解析の中で、母集団の層別化によって引き起こされる交絡を制御する必要性を示しています。

****

## Hailを起動 
モジュールhailをhlとしてロードし、イニシャライズします。

In [1]:
import hail as hl
hl.init()

2022-02-10 12:10:02 WARN  Utils:69 - Your hostname, utmba.local resolves to a loopback address: 127.0.0.1; using 10.31.100.200 instead (on interface en0)
2022-02-10 12:10:02 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address
2022-02-10 12:10:03 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.2
SparkUI available at http://utmba:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.81-edeb70bc789c
LOGGING: writing to /Users/ucdtmhr/Desktop/hail-20220210-1209-0.2.81-edeb70bc789c.log


Hailのステキなアスキーアートが表示されれば準備OKです。  
次に、ノートブック全体で使用するためにいくつかの標準Pythonライブラリをインポートします。

ひとつひとつが、このノートブックの後半で使われる機能をセットアップしています。   
少し冗長に感じるかもしれませんが、以下に解説をいれておきます。


|命令文|意味、用途|
|----|----|
|from hail.plot import show| 後半の図表作成、show(p)で使われる機能を準備しています|
|from pprint import pprint|後半の結果表示で使われます。きれいに整形された出力や表示が可能です|
|hl.plot.output_notebook()|データ可視化ライブラリbokehを利用し、notebookに図表を描画できるようにしています|

In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

***
## サンプルデータの準備と読み込み

### Download public 1000 Genomes data

1000人ゲノムのデータ(ジェノタイピングされたSNPが入った大きなVCFファイル)を約20MBにダウンサンプリングすることによって作成された小さなデータを使用します。 また、別のテキストファイルからのサンプルメタデータとバリアントメタデータを統合していきます。

これらのファイルはHail teamにより、Google Storageでホストされています。    
次の命令を実行することで次のファイルがローカル環境にダウンロードされます。

<img src="images/hail-sample-datas-list.png" width="300">

In [3]:
hl.utils.get_1kg('data/')

2022-02-10 12:10:08 Hail: INFO: 1KG files found


### VCFファイルをインポートします

VCFファイルをHailネイティブなデータ形式である[MatrixTable](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable)にインポートします。  
まず冒頭のhl.import_vcf('data/1kg.vcf.bgz')でインポートし、  
つづきの.write('data/1kg.mt', overwrite=True)でHailのネイティブなファイル形式で書き込んでいます。  
これにより、以降のデータ解析がはるかに高速になります。

In [4]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)

2022-02-10 12:10:17 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-02-10 12:10:26 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 1 partition to data/1kg.mt


次に、今書き込んだファイルを変数mtとして読み込みます。  
任意の名前をつけられますがここでは、'm'atrix 't'ableということでmtとしてみました。

ところで、hlの続きにread_matrix_tableとありますがこれがMatrixTableを読み込む命令です。  
詳しいオプションはHailのドキュメント 
(https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table)  
に記載があります。  
他の機能を探したいときやオプションを知りたいときにはこのあたりを参照します。


<img src="images/RefDoc-read_matrix_table.png" width="500">

In [5]:
mt = hl.read_matrix_table('data/1kg.mt')

### データを見てみましょう

データセットを操作し、一部を確認したり、条件に応じたデータを抽出したり、要約したりといったことは簡単に実行できます。  
この辺りの機能を以下で示していきます。

メソッド[rows](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.rows) は、  
MatrixTableからrowフィールドを取得します。  

つづけて[select](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) を使用することでバリアントを取り出しています。

`select` メソッドでは引数として、テーブル内のフィールド名を参照する文字列や、[Hail Expression](https://hail.is/docs/0.2/hail.expr.Expression.html?#expression)
のいずれかを指定できます。  

ここでは、引数を空白のままにしていますが、そうすると行のキーフィールドである `locus`と` alleles`のみを返してきます。

さらにselectにつづけて`show` メソッドに引数5を指定して使うことで、バリアントの最初の5つを表示させることができます。 


In [6]:
mt.rows().select().show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


もしくは、row_keyという命令を使うとRowのkeyを表示するので同じ結果が得られます。　

In [7]:
mt.row_key.show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


#### 補足 rowフィールドには何が入っているの？？
mt.rows().describe()を実行するとフィールドの概要が表示されます。  

見てみると、rowsにはvariantに関するデータがあり、locus, alleles, rsid, qual, filters, infoなどが含まれていることがわかります。


In [8]:
mt.rows().describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    } 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


### サンプルIDを見てみる
サンプルIDの初めの5個を見てみます。  
sは、mtのcolumnフィールドのkeyで、サンプルIDが格納されています。
columnフィールドにはサンプルに関する情報が格納されます。

In [9]:
mt.s.show(5)

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""


最初のいくつかの遺伝子型を表示させてみます。  

遺伝子型はentriesフィールドに入っていて、  
[entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries)  
を使うことで取り出せます。  

今回は`take`を使い表示させています。  
`take` は最初のn行を収集してきます。  

もしくは、表示形式は異なりますが先ほど使った`show`でも構いません。  

In [10]:
mt.entry.take(5)

[Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 147]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=24, PL=[0, 24, 315]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=23, PL=[0, 23, 230]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[7, 0], DP=7, GQ=21, PL=[0, 21, 270]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=15, PL=[0, 15, 205])]

In [11]:
mt.entry.show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


***
## MatrixTable ?? rowフィールド？？ 整理してみます
ここまでで、VCFファイルをインポートして、hailのネイティブな形式であるMatrixTableにしました。  
また、MatrixTableの中を少し見てみました。  
MatrixTableではVCFファイルにある様々なデータを、計算機で扱いやすいように整理して格納していますが、もう少し詳しくみてみます。  

## MatrixTableの概要を知る
mt.describe()を実行すると概要をみることができます。  

Global fields, Column fields, Row fields, Entry fields, Column key, Row keyがあることがわかります。  

VCFファイルにあった情報がどのフィールドに格納されているか、確認してみましょう。  

Column fieldにはサンプルの情報が、  
Row fieldはバリアントの情報が、  
Entry fieldにはサンプルごとの情報が入っていると思います。


In [12]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    }
----------------------------------------
Entry fields:
    'GT': call
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'PL': array<int32>
----------------------------------------
Colu

##### 実行してみましょう
次の3つのセルの、冒頭にある'#'を削除して実行してみてください。  
それぞれのフィールドの内容が表示されればOKです。

- column field

In [13]:
#mt.cols().show(5)

- row field

In [14]:
#mt.rows().show(5)

- entry field

In [15]:
#mt.entries().show(5)

##### 実行できましたか？？ 

hailではこのように格納されたデータを操作していくことができます。  
ひとつひとつのデータについて検索したり、条件で絞り込んだりすることができます。   

このチュートリアルの後半ではそうした機能を使い、解析を進めていくことをみることができます。

※参考:  
[MatrixTableのCheat sheet](https://hail.is/docs/0.2/_static/cheatsheets/hail_matrix_tables_cheat_sheet.pdf)も便利ですよ。  
また、あとで出てくる[HailのTable(以下 Table)のcheat sheet](https://hail.is/docs/0.2/_static/cheatsheets/hail_tables_cheat_sheet.pdf)もあります。

### columnフィールドへデータを追加

HailのMatrixTableでは遺伝統計研究で重要なアノテーションなど、任意の新たなデータを追加していくことができます。
Columnフィールドはサンプルの表現型、祖先、性別や共変量に関する情報を格納するのに使います。  
RowフィールドはQCまたは分析で使用するための遺伝子や機能的影響などの情報を格納するために使用できます。
ここでは、テキストファイルをもとにMatrixTableに注釈をつける方法を示します。

提供されるファイルには、サンプルID(sample ID)、populationとsuper-population、性別、および2つのシミュレートされた表現型（1つはバイナリ、1つは離散）が含まれています。

このcsvファイルはHailの[import_table](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_table)で、インポートできます。  
これにより、[Table](https://hail.is/docs/0.2/hail.Table.html#hail.Table)オブジェクトが生成されます。  
データには先の手順でDownloadしている、data/1kg_annotations.txtを使います。  
データの型は、impute=Trueを使い推定させています。  
key_by('Sample')により、後ほどMatrixTableとの結合に使うキーを指定しています。

このTableはPandasやRのdataframeのようなものですが、Sparkの恩恵をうけた、マシンのメモリ容量によって制限されないものです。 

In [16]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

2022-02-10 12:10:41 Hail: INFO: Reading table to impute column types
2022-02-10 12:10:42 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


これで読み込めましたが、一応、元のファイルの内容を確認してみます。

In [17]:
!head data/1kg_annotations.txt

Sample	Population	SuperPopulation	isFemale	PurpleHair	CaffeineConsumption
HG00096	GBR	EUR	false	false	4
HG00097	GBR	EUR	true	true	4
HG00098	GBR	EUR	false	false	5
HG00099	GBR	EUR	true	false	4
HG00100	GBR	EUR	true	false	5
HG00101	GBR	EUR	false	true	1
HG00102	GBR	EUR	true	true	6
HG00103	GBR	EUR	false	true	5
HG00104	GBR	EUR	true	false	5


どのようになったか見てみましょう。  
table.describe()で概要を確認できます。

In [18]:
table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Sample': str 
    'Population': str 
    'SuperPopulation': str 
    'isFemale': bool 
    'PurpleHair': bool 
    'CaffeineConsumption': int32 
----------------------------------------
Key: ['Sample']
----------------------------------------


MatrixTableのときと同じように、`show`を使うことでデータを確認できます。

In [19]:
table.show(width=100)

,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",False,False,4
"""HG00097""","""GBR""","""EUR""",True,True,4
"""HG00098""","""GBR""","""EUR""",False,False,5
"""HG00099""","""GBR""","""EUR""",True,False,4
"""HG00100""","""GBR""","""EUR""",True,False,5
"""HG00101""","""GBR""","""EUR""",False,True,1
"""HG00102""","""GBR""","""EUR""",True,True,6
"""HG00103""","""GBR""","""EUR""",False,True,5


次に、このテーブルを使用してMatrixTableのcolumnフィールドにアノテーションを格納してみます。   
まず、既存のcolumnフィールドのスキーマを確認します。

In [20]:
print(mt.col.dtype)

struct{s: str}


または、

In [21]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


MatrixTableの機能である[annotate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) メソッドを使用して、先ほど読み込んだMatrixTableとこのtableを結合します。  
  
ここでは、mt.sにあるsample ID(HG00096など)を使い、tableの中を検索し、ヒットしたものをcolsフィールドにphenoという構造体で格納しています。

In [22]:
mt = mt.annotate_cols(pheno = table[mt.s])

tableにあったPurpleHairなどの情報がMatrixTableでどのようになったか、確認してみましょう。　　

In [23]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [24]:
mt.col.show()

+-----------+------------------+-----------------------+----------------+
| s         | pheno.Population | pheno.SuperPopulation | pheno.isFemale |
+-----------+------------------+-----------------------+----------------+
| str       | str              | str                   |           bool |
+-----------+------------------+-----------------------+----------------+
| "HG00096" | "GBR"            | "EUR"                 |          False |
| "HG00099" | "GBR"            | "EUR"                 |           True |
| "HG00105" | "GBR"            | "EUR"                 |          False |
| "HG00118" | "GBR"            | "EUR"                 |           True |
| "HG00129" | "GBR"            | "EUR"                 |          False |
| "HG00148" | "GBR"            | "EUR"                 |          False |
| "HG00177" | "FIN"            | "EUR"                 |           True |
| "HG00182" | "FIN"            | "EUR"                 |          False |
| "HG00242" | "GBR"            | "EUR"                 |          False |
| "HG00254" | "GBR"            | "EUR"                 |           True |
+-----------+------------------+-----------------------+----------------+

+------------------+---------------------------+
| pheno.PurpleHair | pheno.CaffeineConsumption |
+------------------+---------------------------+
|             bool |                     int32 |
+------------------+---------------------------+
|            False |                         4 |
|            False |                         4 |
|            False |                         4 |
|            False |                         3 |
|            False |                         6 |
|             True |                         2 |
|             True |                         4 |
|            False |                         2 |
|            False |                         1 |
|            False |                         2 |
+------------------+---------------------------+
showing top 10 rows

細かいですが、↑の表記(mt.col)は [HailのStructExpression] (https://hail.is/docs/0.2/hail.expr.StructExpression.html#structexpression)です。  
続けてdescribeのほかselectなども利用可能ですが、MatrixTableでもTableでもなかったりします。  

その出力やオブジェクトが何かを判定するには次のように.show()などを付与せずに実行するとわかります。

In [25]:
mt.col

<StructExpression of type struct{s: str, pheno: struct{Population: str, SuperPopulation: str, isFemale: bool, PurpleHair: bool, CaffeineConsumption: int32}}>

***
## データの操作

### Query functions and the Hail Expression Language

Hailにはデータセットの統計情報を計算し取得するための便利な関数がいくつかあります。  
これらの関数は引数として`Hail Expression`をとります。

まず、先ほど読み込んだtableの統計情報を取得してみましょう。  
[aggregate](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate)
メソッドを使うことで、tableの行を集計てきます。

下のセルで、`counter`は、一意の各要素の出現回数をカウントする集計関数です。  
これを使用して、カウントしたいフィールドを表す`Hail Expression`(ここでは、table.SuperPopulation)を渡すことにより、人口分布を集計することができます。

1. table.aggregateで集計を指示
2. 集計内容はcounter
3. 集計対象はtable.SuperPopulation  
という流れです

In [26]:
pprint(table.aggregate(hl.agg.counter(table.SuperPopulation)))

frozendict({'AFR': 1018, 'AMR': 535, 'EAS': 617, 'EUR': 669, 'SAS': 661})


#### その他の項目でも試行してみましょう

- pprint(table.aggregate(hl.agg.counter(table.Population)))

In [27]:
pprint(table.aggregate(hl.agg.counter(table.Population)))

frozendict({'ACB': 123, 'ASW': 112, 'BEB': 144, 'CDX': 109, 'CEU': 183, 'CHB': 108, 'CHS': 171, 'CLM': 148, 'ESN': 173, 'FIN': 105, 'GBR': 107, 'GIH': 113, 'GWD': 180, 'IBS': 162, 'ITU': 118, 'JPT': 105, 'KHV': 124, 'LWK': 116, 'MSL': 128, 'MXL': 107, 'PEL': 130, 'PJL': 158, 'PUR': 150, 'STU': 128, 'TSI': 112, 'YRI': 186})


- pprint(table.aggregate(hl.agg.counter(table.isFemale)))

In [28]:
pprint(table.aggregate(hl.agg.counter(table.isFemale)))

frozendict({False: 1740, True: 1760})


- pprint(table.aggregate(hl.agg.counter(table.PurpleHair)))

In [29]:
pprint(table.aggregate(hl.agg.counter(table.PurpleHair)))

frozendict({False: 1689, True: 1811})


- pprint(table.aggregate(hl.agg.counter(table.CaffeineConsumption)))

In [30]:
pprint(table.aggregate(hl.agg.counter(table.CaffeineConsumption)))

frozendict({-1: 1, 0: 84, 1: 159, 2: 414, 3: 677, 4: 849, 5: 685, 6: 394, 7: 174, 8: 45, 9: 17, 10: 1})


#### それぞれの統計情報を取得

`stats` を使うと統計情報を取得するように集計内容が変わります。  
たとえばCaffeineConsumptionの分布を見るには、

1. table.aggregateで集計を指示
2. 集計内容はstats
3. 集計対象はtable.CaffeineConsumption  
という流れです

In [31]:
pprint(table.aggregate(hl.agg.stats(table.CaffeineConsumption)))

{'max': 10.0,
 'mean': 3.9837142857142855,
 'min': -1.0,
 'n': 3500,
 'stdev': 1.7021055628070711,
 'sum': 13943.0}


良い感じですね。  

ですが実は、これらは先のVCFファイル由来のデータセット内にあるサンプルをきちんと表せているとは限りません。  
理由は次のとおりです。

tableに.count()を付与することで、tableに登録されているデータの件数が確認できます

In [32]:
table.count()

3500

mtに.count_cols()を付与することで、mtのcolsフィールドに登録されているサンプルのデータ件数を確認できます。  
または、mt.cols().count()でも同じことができます。

In [33]:
mt.count_cols()

284

tableに比べてmtは、そもそもダウンサンプリングしていたものなので1000人ゲノムのデータにしては少ないです。  
そのため、見るべきはMatrixTable mtに付与したアノテーションの方でした。  

[aggregate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) 
を使うことで、MatrixTableにあるサンプルに対するメトリクスを取得することができます。

1. mt.aggregate_colsで、MatrixTableより集計することを指示
2. 集計内容はcounter
3. 集計対象はmt.pheno.SuperPopulation
という流れです

In [34]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

frozendict({'AFR': 76, 'AMR': 34, 'EAS': 72, 'EUR': 47, 'SAS': 55})

同様に、CaffeineConsumptionの統計値を取得してみます。

In [35]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

{'max': 9.0,
 'mean': 4.415492957746479,
 'min': 0.0,
 'n': 284,
 'stdev': 1.577763427465917,
 'sum': 1254.0}


##### その他のツールと何がちがうの？
これらの機能は世間一般でそれほど新しいものではありません。  
そもそもPandasやRのDataFrame、あるいはawkなどのUnixツールで実施してきたことです。  
しかしHailではこのようなインターフェースや言語を用いて、すべてのvariantのセットなどの遥かに大きなデータを分析することができるのです。  


##### 次に、12通りの一位なSNPをそれぞれカウントしてみます。  

それには各バリアントのalternate alleleを取得してから、それぞれのRef/Altペアをカウントしていく必要があります。  
これは、Hailの`counter`機能で実行します。

先の例と同様に、今度はaggregate_rowsを使うことでバリアントの情報が格納されているrowsフィールドの集計ができます。  

1. mt.aggregate_rowsで、MatrixTableのrowsフィールドより集計することを指示
2. 集計内容はcounter
3. 集計対象はhl.Struct(ref=mt.alleles[0], alt=mt.allales[1])です。  
こうすることで出現したRef/Altをカウントしていくことができます。  


mt.alleles[0]やmt.alleles[1]は下の画像のとおり、Reference/Alternative それぞれのアレルを取り出します。


<img src="images/ref-alt-alleles.png" width="500">

では、snpのカウントを実行してみます。  
結果は変数snp_countsに代入し、直後に表示させています。  

In [36]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)


frozendict({Struct(ref='A', alt='C'): 451, Struct(ref='A', alt='G'): 1929, Struct(ref='A', alt='T'): 75, Struct(ref='C', alt='A'): 494, Struct(ref='C', alt='G'): 150, Struct(ref='C', alt='T'): 2418, Struct(ref='G', alt='A'): 2367, Struct(ref='G', alt='C'): 111, Struct(ref='G', alt='T'): 477, Struct(ref='T', alt='A'): 77, Struct(ref='T', alt='C'): 1864, Struct(ref='T', alt='G'): 466})


PythonのCounterクラスを使用して、降順で一覧表示してみます。  
collectionsのCounterは、任意のリストにある各要素の出現頻度を扱うのに便利な機能のひとつです。  
most_common()で頻度が多い順にリストすることができます。  

In [37]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

[(Struct(ref='C', alt='T'), 2418),
 (Struct(ref='G', alt='A'), 2367),
 (Struct(ref='A', alt='G'), 1929),
 (Struct(ref='T', alt='C'), 1864),
 (Struct(ref='C', alt='A'), 494),
 (Struct(ref='G', alt='T'), 477),
 (Struct(ref='T', alt='G'), 466),
 (Struct(ref='A', alt='C'), 451),
 (Struct(ref='C', alt='G'), 150),
 (Struct(ref='G', alt='C'), 111),
 (Struct(ref='T', alt='A'), 77),
 (Struct(ref='A', alt='T'), 75)]

この小さなデータセットからでも、生物学的なものを実際に発見できてますね！  
これらの頻度はペアになっていることがわかります。 C / TとG / Aは実際には同じ突然変異であり、反対側の鎖から見ただけです。 同様に、T / AとA / Tは反対の鎖の同じ突然変異です。 C / TとA / TのSNPの頻度には30倍の違いがあります。 どうして？

(utメモ: ピリミジン(C,T/U)、プリン(A,G)間の変異をtransition, ピリミジン-プリン間の変異をtransversionと呼び、後者は発生しにくい)

同じPython、R、およびUnixツールでもこの作業を行うことができますが、データの大きさの壁にぶつかり始めています。  
最新の[gnomAD release](https://gnomad.broadinstitute.org/) は、約2億5000万のバリアントを公開しています。   
そして、それはもはや単一のコンピュータのメモリに収まりません。  

遺伝子型はどうですか？   
Hailは、データセット内のすべての遺伝子型のコレクションを検索できます。これは、小さなデータセットでも大きくなっています。 284のサンプルと10,000のバリアントにより、1,000万の固有の遺伝子型が生成されます。 gnomADデータセットには、約5兆の固有の遺伝子型があります。

#### DPのヒストグラムをプロットしてみます
hailのplot関数では、hailのフィールドを引数として使用できるため、次のように直接DPフィールドを渡すことができます。 引数 rangeとbinsが設定されていない場合、この関数はフィールドの最小値と最大値に基づいて範囲を計算し、binはデフォルトの50を使用します。

range外のものは、'Outliters Above'や'Outliters Below'として扱われます。

In [38]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)

***
## Quality Control

QCは時間を費やす必要がある場所です。反復する必要があるプロセスですが、手順は毎度異なります。QCには「これをやればOK」というソリューションは無いとも言えます。Broadでも新しいグループのサンプルを得るたびに、新しい方法を見つけています。しかし、オープンサイエンスに基づいてこのQCについて話し合うことで、ベストプラクティスを確立していくことができます。

### Sample QC
QCの基礎はデータセットの詳細を理解することにあります。  
Hailはこれを、[sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) 機能によってより簡単に実行できるようにしています。
これは、データセットの理解に有用なメトリクスを生成し、それらをcolumn フィールドに格納します。

まず、現時点でのcolumnフィールドの詳細を確認しておきます。  
この時点ではQCに関する項目はありません。

In [39]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


次に、sample_qcを実行します。  
これによりdp_statsなどの統計値が追加されます。

In [40]:
mt = hl.sample_qc(mt)

確認してみましょう。
sample_qcが追加されていることがわかります。

In [41]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
            n_het: int64, 
            n_hom_var: int64, 
            n_non_ref: int64, 
            n_singleton: int64, 
            n_snp: int64, 
            n_insertio

sample_qcで得られた値をグラフにプロットしてみましょう。  
hail (hl) のplot.histogramという機能でヒストグラムを描画できます。  
ここではcall rateを対象にし、0.88から1の範囲をプロットしています。  

In [42]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)
# Call RateはGenoypingの成功率。
# call_rate (float64) – Fraction of calls not missing or filtered. Equivalent to n_called divided by count_rows().

同様に、gq_stats.meanを10-70の範囲でプロットします。

In [88]:
mt.GQ.show()

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,GQ,GQ,GQ
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",12,24,23
1:1563691,"[""T"",""G""]",NA,6,4
1:1707740,"[""T"",""G""]",67,63,57
1:2284195,"[""T"",""C""]",3,90,35
1:2779043,"[""T"",""C""]",33,28,6
1:2944527,"[""G"",""A""]",6,60,NA
1:3761547,"[""C"",""A""]",12,15,24


In [43]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)
# GQ: Quality of the assigned genotype. 上限は99

多くの場合、これらのメトリックには相関があります。

hl.plot.scatterでdp_stats.meanとcall_rateの散布図を描いてみます。  

In [89]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)
# DP: filtered depthの平均値とcall_rateの散布図
# DPが高いとcall_rateが高いし、DPが低いとcall_rateは低くなる相関がわかる

データセットから外れ値を削除すると、通常、関連解析の結果が向上します。 任意のカットオフを作成し、それらを使用してフィルタリングできます。

ここではdp_stats.meanが4以上でかつ、call_rateが0.97以上であるものにフィルターしています。  
また、どれくらいフィルターされているかを表示しています。

In [45]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

After filter, 250/284 samples remain.


先ほど実行した3つのグラフを再描画してみましょう。

In [46]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [47]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)

In [48]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

次はGenotypeのQCです。  

本来あるべき場所にないところを読み取っている遺伝子型を除外することをおすすめします:    
10%を超えるalternate読み取りを伴う、REF alleleのホモ接合型,  
10%を超えるreference読み取りを伴う、ALT alleleのホモ接合型,  
または、1:1に近いref/altのバランスにない、ヘテロ接合型,  
はエラーである可能性があります。  


In a low-depth dataset like 1KG, it is hard to detect bad genotypes using this metric, since a read ratio of 1 alt to 10 reference can easily be explained by binomial sampling. However, in a high-depth dataset, a read ratio of 10:100 is  a sure cause for concern!


1KGのような低深度のデータセットでは、このメトリックを使用して悪い遺伝子型を検出することは困難です。  
これは、1alt対10refの読み取り比率が二項サンプリングによって簡単に説明できるためです。   
ただし、高深度のデータセットでは、読み取り比が10：100であることは確かに懸念の原因です。

????

以下、メモ
```
# AD: allele depth. フィルターなし状態でのカバレッジ。REF, ALTと２つの値があり、AD[1]はALTの値。
# hl.sum(mt.AD)は、REF,ALTを足した値。
# filter_condition_abの結果は、True or NA or False
# is_hom_ref: True if the call has no alternate alleles.
# is_het: True if the call contains two different alleles.
# is_hom_var: True if the call contains identical alternate alleles.
# 
# abが小さい領域はrefであるはず。大きい領域はaltであるはず。hetはその間。それ以外がエラーと仮定している。
#
# fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
# ここではTrueだった箇所の割合を算出。0.03596...
# 
# mt = mt.filter_entries(filter_condition_ab)
# ここで先のfilterを適用。entriesをカウントすると、2,719,750 -> 97,808になっている。
```

フィルターを実行する前に、mt.entriesを表示しておきます。  
また、あとで比較に使いたいので  
mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()  
を実行しておきます。

In [49]:
mt.entries().show()

2022-02-10 12:11:26 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [514]        | [1.02e-01]     

In [50]:
mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()

,,,
locus,alleles,s,GT
locus<GRCh37>,array<str>,str,call
1:9780836,"[""T"",""G""]","""HG00096""",0/0
1:9780836,"[""T"",""G""]","""HG00099""",NA
1:9780836,"[""T"",""G""]","""HG00105""",NA
1:9780836,"[""T"",""G""]","""HG00118""",0/0
1:9780836,"[""T"",""G""]","""HG00129""",NA
1:9780836,"[""T"",""G""]","""HG00148""",0/0
1:9780836,"[""T"",""G""]","""HG00254""",0/0
1:9780836,"[""T"",""G""]","""HG00271""",NA


In [51]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

Filtering 3.60% entries out of downstream analysis.


上記はちょっと複雑そうに見えますので、少し細かくみてみましょう。

- ab = md.AD[1] / hl.sum(mt.AD) の行

mt.AD[1]は、下記のmt.AD.show()で表示されるADのalt側の数値です。  
hl.sum(mt.AD)は、ref/altの合計値です。


In [52]:
mt.AD.show(3)

,,,
,,'HG00096','HG00099'
locus,alleles,AD,AD
locus<GRCh37>,array<str>,array<int32>,array<int32>
1:904165,"[""G"",""A""]","[4,0]","[8,0]"
1:909917,"[""G"",""A""]","[4,0]","[8,0]"
1:986963,"[""C"",""T""]","[3,0]","[1,0]"


In [53]:
mt.AD[1].show(3)

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,,,
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",0,0,0
1:909917,"[""G"",""A""]",0,0,0
1:986963,"[""C"",""T""]",0,0,0


In [54]:
hl.sum(mt.AD).show()

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,,,
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",4,8,8
1:909917,"[""G"",""A""]",4,8,5
1:986963,"[""C"",""T""]",3,1,1
1:1563691,"[""T"",""G""]",NA,2,2
1:1707740,"[""T"",""G""]",5,6,7
1:2252970,"[""C"",""T""]",4,NA,7
1:2284195,"[""T"",""C""]",1,7,4


- filter_condition_abの行  
```
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
```

filter_condition_abの結果は、True or NA or False で返されます。  
&は論理積(AND), | "パイプ"は論理和(OR)です。  

mt.GTに続く命令は、以下のような条件でTrue or Falseを返すものです。  
```
is_hom_ref: True if the call has no alternate alleles.
is_het: True if the call contains two different alleles.
is_hom_var: True if the call contains identical alternate alleles.
```
これに、abの値によるフィルターを組み合わせています。



filter_condition_abを見てみましょう

In [55]:
filter_condition_ab.show(30,n_cols=100)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,'HG00096','HG00099','HG00105','HG00118','HG00129','HG00148','HG00254','HG00271','HG00332','HG00335','HG00421','HG00452','HG00472','HG00534','HG00583','HG00598','HG00619','HG00623','HG00657','HG00663','HG00704','HG00705','HG00733','HG00881','HG01052','HG01070','HG01075','HG01164','HG01174','HG01241','HG01248','HG01256','HG01275','HG01284','HG01334','HG01348','HG01396','HG01443','HG01491','HG01498','HG01537','HG01572','HG01606','HG01623','HG01630','HG01783','HG01784','HG01790','HG01799','HG01801','HG01806','HG01812','HG01813','HG01817','HG01848','HG01849','HG01857','HG01863','HG01874','HG01915','HG01924','HG01965','HG01970','HG01991','HG02010','HG02020','HG02054','HG02086','HG02087','HG02116','HG02122','HG02130','HG02152','HG02154','HG02165','HG02232','HG02236','HG02250','HG02259','HG02298','HG02345','HG02351','HG02363','HG02373','HG02383','HG02384','HG02386','HG02388','HG02389','HG02397','HG02419','HG02462','HG02464','HG02497','HG02521','HG02561','HG02574','HG02580','HG02595','HG02603'
locus,alleles,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
locus<GRCh37>,array<str>,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
1:904165,"[""G"",""A""]",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1:909917,"[""G"",""A""]",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1:986963,"[""C"",""T""]",True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,NA,True,True,NA,True,True,True,NA,True,True,True,True,True,NA,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,NA,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1:1563691,"[""T"",""G""]",NA,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,NA,False,True,True,True,NA,False,True,True,True,True,True,True,NA,True,True,True,True,NA,True,True,True,True,False,True,True,True,True,True,True,True,True,False,True,True,False,False,True,True,True,True,True,True,True,False,True,True,True,True,False,False,True,True,True,True,NA,True,True,True,True,False,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1:1707740,"[""T"",""G""]",True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,

1:9780836 は True, NA, Falseが都合よく混ざっているので確認に良さそうです。  

- fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))  

```
fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
```

ここではTrueだった箇所の割合を算出しています。  
結果を見てみましょう。

In [56]:
fraction_filtered

0.03596212887213898

- mt = mt.filter_entries(filter_condition_ab)
```
mt = mt.filter_entries(filter_condition_ab)
```

ここではmtのentriesフィールドにfilter_condition_abをフィルターとして適用しています。

どのようにフィルターされたのか、1:9780836を探して見てみましょう。

In [57]:
mt.select_entries(mt.GT).show()

,,,,,
,,'HG00096','HG00099','HG00105','HG00118'
locus,alleles,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call
1:904165,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:909917,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:986963,"[""C"",""T""]",0/0,0/0,0/0,0/0
1:1563691,"[""T"",""G""]",NA,0/0,0/0,0/0
1:1707740,"[""T"",""G""]",0/1,0/1,0/1,0/0
1:2252970,"[""C"",""T""]",0/0,NA,0/0,0/0
1:2284195,"[""T"",""C""]",1/1,0/1,0/1,0/1


染色体上の位置を定義するのは次のような操作になります。

In [58]:
l1 = hl.Locus.parse('1:9780836')

In [59]:
l1

Locus(contig=1, position=9780836, reference_genome=GRCh37)

これを使ってみます。

In [60]:
mt.filter_rows(mt.locus == l1).show()

,
locus,alleles
locus<GRCh37>,array<str>
1:9780836,"[""T"",""G""]"


ちょっとよくわからない結果をもらってしまいました。  
しかしこれは確認するともっともなところです。  
mt.filter_rows(mt.locus == l1) 単体で実行してみると、それがMatrixTableなのかHail Tableなのかを確認できます。

In [61]:
mt.filter_rows(mt.locus ==l1)

これはMatrixTableでした。確認したかったのはentriesフィールドのフィルター具合なので、次のように実行します。 

In [62]:
mt.filter_rows(mt.locus == l1).entries().show()

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
| 1:9780836     | ["T","G"]  | NA   | 3.41e+03 | NA       | [640]        |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
| [1.64e-01]     |    3896 |         -5.18e+01 |             2.60e+00 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
|    6944 |   False | 3.20e+03 |                  NA |            -1.53e-02 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [508]        | [1.30e-01]     

ちょっと出力が多すぎるのでselectを使って結果をGTのみに限定します。

In [63]:
mt.filter_rows(mt.locus == l1).entries().select("GT").show()

,,,
locus,alleles,s,GT
locus<GRCh37>,array<str>,str,call
1:9780836,"[""T"",""G""]","""HG00096""",0/0
1:9780836,"[""T"",""G""]","""HG00254""",0/0
1:9780836,"[""T"",""G""]","""HG00332""",0/0
1:9780836,"[""T"",""G""]","""HG00335""",0/0
1:9780836,"[""T"",""G""]","""HG00452""",0/0
1:9780836,"[""T"",""G""]","""HG00472""",0/0
1:9780836,"[""T"",""G""]","""HG00534""",1/1
1:9780836,"[""T"",""G""]","""HG00583""",0/0


ちなみに、l1のところの表記はこのようにしてもOKです。  

HG00096の次がHG00254となっており、 NAやFalseとなっていたサンプルがフィルターされていることが確認できました。

In [64]:
mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()

,,,
locus,alleles,s,GT
locus<GRCh37>,array<str>,str,call
1:9780836,"[""T"",""G""]","""HG00096""",0/0
1:9780836,"[""T"",""G""]","""HG00254""",0/0
1:9780836,"[""T"",""G""]","""HG00332""",0/0
1:9780836,"[""T"",""G""]","""HG00335""",0/0
1:9780836,"[""T"",""G""]","""HG00452""",0/0
1:9780836,"[""T"",""G""]","""HG00472""",0/0
1:9780836,"[""T"",""G""]","""HG00534""",1/1
1:9780836,"[""T"",""G""]","""HG00583""",0/0


### Variant QC
[variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) も同様に、様々な統計情報を計算し格納します。  
それらもプロットしたりフィルターするのに使用できます。

In [65]:
mt = hl.variant_qc(mt)

rowsフィールドに格納されます。確認してみましょう。

In [66]:
mt.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        rsid: str, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            BaseQRankSum: float64, 
            ClippingRankSum: float64, 
            DP: int32, 
            DS: bool, 
            FS: float64, 
            HaplotypeScore: float64, 
            InbreedingCoeff: float64, 
            MLEAC: array<int32>, 
            MLEAF: array<float64>, 
            MQ: float64, 
            MQ0: int32, 
            MQRankSum: float64, 
            QD: float64, 
            ReadPosRankSum: float64, 
            set: str
        }, 
        variant_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 

In [67]:
mt.rows().variant_qc.show()

+---------------+------------+--------------------------+
| locus         | alleles    | variant_qc.dp_stats.mean |
+---------------+------------+--------------------------+
| locus<GRCh37> | array<str> |                  float64 |
+---------------+------------+--------------------------+
| 1:904165      | ["G","A"]  |                 7.57e+00 |
| 1:909917      | ["G","A"]  |                 6.27e+00 |
| 1:986963      | ["C","T"]  |                 6.02e+00 |
| 1:1563691     | ["T","G"]  |                 6.84e+00 |
| 1:1707740     | ["T","G"]  |                 8.22e+00 |
| 1:2252970     | ["C","T"]  |                 6.93e+00 |
| 1:2284195     | ["T","C"]  |                 7.90e+00 |
| 1:2779043     | ["T","C"]  |                 5.28e+00 |
| 1:2944527     | ["G","A"]  |                 7.98e+00 |
| 1:3761547     | ["C","A"]  |                 6.97e+00 |
+---------------+------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  3.78e+00 |                1.00e+00 |
|                  3.85e+00 |                1.00e+00 |
|                  3.85e+00 |                1.00e+00 |
|                  4.36e+00 |                1.00e+00 |
|                  4.21e+00 |                1.00e+00 |
|                  4.52e+00 |                1.00e+00 |
|                  4.27e+00 |                1.00e+00 |
|                  2.81e+00 |                1.00e+00 |
|                  4.77e+00 |                1.00e+00 |
|                  3.60e+00 |                1.00e+00 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                2.10e+01 |                 2.93e+01 |
|                2.30e+01 |                 1.90e+01 |
|                1.90e+01 |                 1.91e+01 |
|                2.00e+01 |                 2.02e+01 |
|                2.60e+01 |                 3.61e+01 |
|                2.40e+01 |                 2.07e+01 |
|                2.20e+01 |                 3.79e+01 |
|                1.50e+01 |                 2.58e+01 |
|                3.10e+01 |                 3.84e+01 |
|                2.40e+01 |                 2.81e+01 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  2.32e+01 |                3.00e+00 |
|                  1.20e+01 |                3.00e+00 |
|                  1.19e+01 |                3.00e+00 |
|                  1.39e+01 |                2.00e+00 |
|                  2.67e+01 |                3.00e+00 |
|                  1.40e+01 |                2.00e+00 |
|                  2.99e+01 |                0.00e+00 |
|                  2.32e+01 |                3.00e+00 |
|                  3.08e+01 |                3.00e+00 |
|                  1.61e+01 |                3.00e+00 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 float64 | array<int32>  | array<float64>      |
+-------------------------+---------------+---------------------+
|                9.90e+01 | [437,57]      | [8.85e-01,1.15e-01] |


variant_qcが追加されたのが確認できたと思います。  

今回のデータセットについてはフィルタリングをする必要はありませんが、通常ほとんどのデータセットでは慎重なQCが必要です。  
[filter_rows](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_rows) が役にたつでしょう。

***
## Let's do a GWAS!

まず、次のようなバリアントに絞り込みます:  

 - common( cutoff 1%)
 - シーケンスエラーを示唆するほどハーディーワインベルク平衡[Hardy-Weinberg equilibrium](https://en.wikipedia.org/wiki/Hardy%E2%80%93Weinberg_principle) から遠くない  


まず、AF[1] (allele frequency for each ALT allele) が0.01以上であるようにフィルターします。

In [68]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

次に、p_value_hwe (float64, p-value from two-sided test of Hardy-Weinberg equilibrium.)が 10^-6 以上であるものにフィルターします。

In [69]:
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

現時点でのSample数とVariant数を確認します。

In [70]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

Samples: 250  Variants: 7774


15%カットされています(最初は10000以上ありました)。ダウンサンプルする段階ですでに通常のデータセットよりcommon variantが含まれるようにしているので、この結果は一般的ではないかもしれません。

Hailでの関連解析では、columnフィールドのサンプルの表現型や共変量を使います。  
すでに関心のある表現型をデータセットにいれているので、それを使います。  


以下ではlinear_regression_rows[https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows] を使い、  
CaffeineConsumptionを使ったテストを行っています。

GT.n_alt_alleles: Alternate alleleの値。0,1,2 or NA   
結果にはp_valueなどが含まれていきます。

In [71]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])
gwas.row.describe()


2022-02-10 12:12:52 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


線形回帰により、beta、標準誤差(standard error )、t統計量、およびp値の新しい行フィールドが追加されていることがわかります。

Hailでは可視化を簡単に実行できます。  
[Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) をつくってみましょう！

In [72]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

ちょっとよくわからないのが出てきましたね...  

[Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q–Q_plot) を使ってgwasが適切だったのか確認してみましょう。

In [73]:
p = hl.plot.qq(gwas.p_value)
show(p)

2022-02-10 12:12:58 Hail: INFO: Ordering unsorted dataset with network shuffle


### Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there's a confounder.

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. 

The [linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows) function can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don't have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) function produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) function does the same, using HWE-normalized genotypes for the PCA.

### 交絡！？

観測されたp値は、すぐに期待値から外れています。私たちのデータセット内のすべてのSNPは、カフェインの消費に因果関係があるか（ありそうもない）、または交絡因子があります。

今になって明かしますが、実際にはサンプルの祖先を使用してこの表現型をシミュレートしていました。  
これは、表現型の分布の層別化につながります。  
解決策は、回帰の共変量として祖先を含めることです。  

[linear_regression_rows]（https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows） 関数は、共変量として使用する列フィールドを取ることもできます。  
報告された祖先でサンプルにすでに注釈を付けましたが、人為的ミスがあるかもしれないのでこのラベルを懐疑的にとらえてみます。  
ゲノムにはその問題はありません！  
報告された祖先を使用する代わりに、計算された主成分をモデルに含めることにより、遺伝的祖先を使用します。  

[pca]（https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca） 関数は、固有値をリストとして生成し、サンプルのPCをテーブルに出力します。  
また、オプションでバリアントの負荷値を生成することもできます。  
[hwe_normalized_pca]（https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca） 関数は、PCAにHWEで正規化された遺伝子型を使用して同じことを行います。 

#### hwe_normalized_pca
Run principal component analysis (PCA) on the Hardy-Weinberg-normalized genotype call matrix.  
hwe_normalized_pcaのreturnは次の通り。  
(list of float, Table, Table) – List of eigenvalues, table with column scores, table with row loadings.

examples: 
```
eigenvalues, scores, loadings = hl.hwe_normalized_pca(dataset.GT, k=5)
```
    

In [74]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

2022-02-10 12:13:04 Hail: INFO: hwe_normalize: found 7766 variants after filtering out monomorphic sites.
2022-02-10 12:13:07 Hail: INFO: pca: running PCA with 10 components... + 1) / 1]


固有値を見てみます。

In [75]:
pprint(eigenvalues)

[18.08411146784072,
 9.984076405601828,
 3.5406872298059473,
 2.6555981083901252,
 1.596852701724399,
 1.5405241027955308,
 1.5077135041162126,
 1.4744976712480333,
 1.4676905390347392,
 1.4461994473306523]


主成分スコアを見てみます。

In [76]:
pcs.show(5, width=100)

,
s,scores
str,array<float64>
"""HG00096""","[1.22e-01,2.81e-01,-1.10e-01,-1.27e-01,6.68e-02,3.29e-03,-2.26e-02,4.26e-02,-9.30e-02,1.83e-01]"
"""HG00099""","[1.14e-01,2.89e-01,-1.06e-01,-6.78e-02,4.72e-02,2.87e-02,5.28e-03,-1.57e-02,1.75e-02,-1.98e-02]"
"""HG00105""","[1.09e-01,2.79e-01,-9.95e-02,-1.06e-01,8.79e-02,1.44e-02,2.80e-02,-3.38e-02,-1.08e-03,2.25e-02]"
"""HG00118""","[1.26e-01,2.95e-01,-7.58e-02,-1.08e-01,1.76e-02,7.91e-03,-5.25e-02,3.05e-02,2.00e-02,-7.78e-02]"
"""HG00129""","[1.06e-01,2.86e-01,-9.69e-02,-1.15e-01,1.03e-02,2.65e-02,-8.51e-02,2.49e-02,5.67e-02,-8.31e-03]"


サンプルごとに主成分が得られたので、それらをプロットすることができます。   
人類の歴史は、遺伝子データセットに強い影響を及ぼします。   
たった50MBのシーケンスデータセットを使用しても、主要な人口集団を回復できます。 

得られた値をMatrixTableへ追加します。  

In [77]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

確認してみます。右のほうにスクロールすると、scoresがあると思います。

In [78]:
mt.cols().show(3)

2022-02-10 12:13:14 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2022-02-10 12:13:19 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]


+-----------+------------------+-----------------------+----------------+
| s         | pheno.Population | pheno.SuperPopulation | pheno.isFemale |
+-----------+------------------+-----------------------+----------------+
| str       | str              | str                   |           bool |
+-----------+------------------+-----------------------+----------------+
| "HG00096" | "GBR"            | "EUR"                 |          False |
| "HG00099" | "GBR"            | "EUR"                 |           True |
| "HG00105" | "GBR"            | "EUR"                 |          False |
+-----------+------------------+-----------------------+----------------+

+------------------+---------------------------+-------------------------+
| pheno.PurpleHair | pheno.CaffeineConsumption | sample_qc.dp_stats.mean |
+------------------+---------------------------+-------------------------+
|             bool |                     int32 |                 float64 |
+------------------+---------------------------+-------------------------+
|            False |                         4 |                4.57e+00 |
|            False |                         4 |                8.25e+00 |
|            False |                         4 |                6.72e+00 |
+------------------+---------------------------+-------------------------+

+--------------------------+------------------------+------------------------+
| sample_qc.dp_stats.stdev | sample_qc.dp_stats.min | sample_qc.dp_stats.max |
+--------------------------+------------------------+------------------------+
|                  float64 |                float64 |                float64 |
+--------------------------+------------------------+------------------------+
|                 2.35e+00 |               1.00e+00 |               2.00e+01 |
|                 3.72e+00 |               1.00e+00 |               2.90e+01 |
|                 2.95e+00 |               1.00e+00 |               2.20e+01 |
+--------------------------+------------------------+------------------------+

+-------------------------+--------------------------+------------------------+
| sample_qc.gq_stats.mean | sample_qc.gq_stats.stdev | sample_qc.gq_stats.min |
+-------------------------+--------------------------+------------------------+
|                 float64 |                  float64 |                float64 |
+-------------------------+--------------------------+------------------------+
|                2.25e+01 |                 2.25e+01 |               0.00e+00 |
|                3.76e+01 |                 2.81e+01 |               0.00e+00 |
|                3.12e+01 |                 2.58e+01 |               0.00e+00 |
+-------------------------+--------------------------+------------------------+

+------------------------+---------------------+--------------------+
| sample_qc.gq_stats.max | sample_qc.call_rate | sample_qc.n_called |
+------------------------+---------------------+--------------------+
|                float64 |             float64 |              int64 |
+------------------------+---------------------+--------------------+
|               9.90e+01 |            9.79e-01 |              10653 |
|               9.90e+01 |            9.89e-01 |              10757 |
|               9.90e+01 |            9.95e-01 |              10827 |
+------------------------+---------------------+--------------------+

+------------------------+----------------------+---------------------+
| sample_qc.n_not_called | sample_qc.n_filtered | sample_qc.n_hom_ref |
+------------------------+----------------------+---------------------+
|                  int64 |                int64 |               int64 |
+------------------------+----------------------+---------------------+
|                    226 |                    0 |                6283 |
|                    122 |                    0 |                6079 |
|                     52 |                    0 |                6238 |
+----------------

主成分スコアをプロットしてみます。  
ラベルにはannotationのテキストファイルにあったもの由来の、SuperPopulationを使用しています。

In [79]:
p = hl.plot.scatter(mt.scores[0], 
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

線形回帰に戻りましょう。  共変量に性別、主成分 PC1, PC2, PC3を使います。

In [80]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2022-02-10 12:13:25 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...


まず、Q-Q plotをつくります。

In [81]:
p = hl.plot.qq(gwas.p_value)
show(p)

2022-02-10 12:13:26 Hail: INFO: Ordering unsorted dataset with network shuffle1]


いいですね！  
この形状は、よくコントロールされたものによると思います。  
マンハッタンプロットを描いてみましょう。

In [82]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

私たちはcaffeine consumption locusを見つけました！  

(原文のまま:笑)
Now simply apply Hail's Nature paper function to publish the result.   
Just kidding, that function won't land until Hail 1.0!

***
## Rare variant analysis

Here we'll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).

ここでは、Hailのexpression languageを使用して、行フィールドと列フィールドの任意のプロパティでグループ化およびカウントする方法を示します。   
Hailは、シーケンスカーネルアソシエーションテスト（SKAT）も実装しています。 

In [83]:
entries = mt.entries()
# MatrixTableからentriesフィイールドをHail tableとして取り出す

results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))

# SuperPopulation, 染色体番号でgroup化し、ヘテロである数をカウント

In [84]:
results.show(10)

2022-02-10 12:13:35 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,,
pop,chromosome,n_het
str,str,int64
"""AFR""","""1""",11039
"""AFR""","""10""",7123
"""AFR""","""11""",6777
"""AFR""","""12""",7016
"""AFR""","""13""",4650
"""AFR""","""14""",4262
"""AFR""","""15""",3847
"""AFR""","""16""",4564


We use the [MatrixTable.entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries) method to convert our matrix table to a table (with one row for each sample for each variant). In this representation, it is easy to aggregate over any fields we like, which is often the first step of rare variant analysis.

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?


ここでは、[MatrixTable.entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries) メソッドを使用して、マトリックステーブルをテーブル(Hail Table)に変換しました（サンプルごとに1行あります。 各バリアント）。 この表現では、好きなフィールドをより簡単に集計できます。これは、レアバリアント分析のよくある最初のステップです。

マイナーアレルfrequencyビンと髪の色でグループ化し、平均GQを計算したい場合はどうなりますか？

In [85]:
entries = entries.annotate(maf_bin = hl.if_else(entries.info.AF[0]<0.01, "< 1%", 
                             hl.if_else(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.pheno.PurpleHair)
      .aggregate(mean_gq = hl.agg.stats(entries.GQ).mean, 
                 mean_dp = hl.agg.stats(entries.DP).mean))

# maf_binを追加。 info.AF[0]が< 0.01なら1%, など
# maf_binごと、purple_haiirごとにgroup化し、
# gq, dqの平均値を算出

In [86]:
results2.show()

2022-02-10 12:13:41 Hail: INFO: Ordering unsorted dataset with network shuffle1]


,,,
af_bin,purple_hair,mean_gq,mean_dp
str,bool,float64,float64
"""1%-5%""",False,2.48e+01,7.43e+00
"""1%-5%""",True,2.46e+01,7.47e+00
"""< 1%""",False,2.35e+01,7.55e+00
"""< 1%""",True,2.35e+01,7.53e+00
""">5%""",False,3.70e+01,7.65e+00
""">5%""",True,3.73e+01,7.70e+00


We've shown that it's easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

 - Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint
 - Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease
 
 
 いくつかの任意の統計によって簡単に集計できることを示しました。 この特定の例は、特に有用な情報を提供しない場合がありますが、この同じパターンを使用して、まれな変動の影響を検出できます。

- 遺伝子ごとの機能的制約を推定するために、機能的カテゴリー（同義、ミスセンス、または機能喪失）ごとに遺伝子ごとのヘテロ接合遺伝子型の数を数えます
- 疾患に関与する遺伝子を検出するために、症例および対照における遺伝子あたりのシングルトン機能喪失変異の数を数える

***
## Epilogue

おつかれさまでした！  
最初のtutorialであるこのGWAS tutorialが終わりました。
さらなるHailのAPIと機能の詳細については、他のチュートリアルをご覧ください。 
追加のHail関数のドキュメントについては、[Python API](https://hail.is/docs/0.2/api.html#python-api) を確認してください。 
自分の研究にHailを使用している場合は、[Zulip chat](https://hail.zulipchat.com) や [discussion forum](https://discuss.hail.is)でご意見をお聞かせください。

参考までに、今日のワークフローを1つのセルにまとめたものを次に示します。

In [87]:
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

mt = hl.read_matrix_table('data/1kg.mt')
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2022-02-10 12:13:42 Hail: INFO: Reading table to impute column types
2022-02-10 12:13:42 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)
2022-02-10 12:13:45 Hail: INFO: hwe_normalize: found 9087 variants after filtering out monomorphic sites.
2022-02-10 12:13:48 Hail: INFO: pca: running PCA with 10 components... + 1) / 1]
2022-02-10 12:13:56 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...
